In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
import weather_forecast_utils
reload(weather_forecast_utils)
from weather_forecast_utils import *

In [2]:
gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')

,YYYY,day,height,DD,MM,geometry
0,2020,124,200.0,3,5,POINT (908589.787 5583661.078)
1,2022,365,235.0,31,12,POINT (895096.419 5647160.622)
2,2020,357,1121.0,22,12,POINT (929725.38 5777738.779)
3,2021,284,1100.0,11,10,POINT (812621.05 5570984.636)
4,2020,159,1973.0,7,6,POINT (770547.743 5635238.603)
...,...,...,...,...,...,...
1055,2021,258,185.0,15,9,POINT (920757.969 5586289.335)
1056,2018,39,1134.0,8,2,POINT (948148.27 5782060.999)
1057,2017,130,449.0,10,5,POINT (1000447.444 5575959.745)
1058,2019,196,1441.0,15,7,POINT (791358.972 5633673.361)


---

In [ ]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index+1)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)

In [3]:
target = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')
target.isna().sum()

YYYY        0
day         0
height      0
DD          0
MM          0
geometry    0
dtype: int64

In [23]:
target['DD'] = target.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
target['MM'] = target.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)
target.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson', index=False)

---

In [ ]:
positives = dict()
negatives = dict()

for y in range(2016, 2025):
    positives[y] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/output_weather_kriging/{y}.geojson')
    negatives[y] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/negative_output_weather_kriging/{y}.geojson')

In [ ]:
for y in range(2016, 2025):
    save_clean_data(pd.DataFrame(positives[y]), "positives_"+str(y), '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')
    save_clean_data(pd.DataFrame(negatives[y]), "negatives_"+str(y), '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')

In [82]:
merged = dict()

positives = dict()
negatives = dict()

for y in range(2016, 2025):
    positives[y] = pd.read_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/database/cache/positives_{y}.csv')
    negatives[y] = pd.read_csv(f'/nfs/home/genovese/thesis-wildfire-genovese/database/cache/negatives_{y}.csv')

for y in positives.keys():
    merged[y] = pd.concat([positives[y], negatives[y]], ignore_index=True)

In [ ]:
from shapely import wkt

for y, gdf in merged.items():
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    save_clean_data(gpd.GeoDataFrame(gdf, geometry='geometry', crs="EPSG:3857"), f'meteo_{y}', '/nfs/home/genovese/thesis-wildfire-genovese/database')

File meteo_2016 already saved
File has been overwritten
File meteo_2017 already saved
File has been overwritten
File meteo_2018 already saved
File has been overwritten
File meteo_2019 already saved
File has been overwritten
File meteo_2020 already saved
File has been overwritten
File meteo_2021 already saved
File has been overwritten
File meteo_2022 already saved
File has been overwritten
File meteo_2023 already saved
File has been overwritten
File meteo_2024 already saved
File has been overwritten


---